# Mini projet 2: La construction des matrices de substitution

Le but du mini projet est de créer des matrices de substitution spécifiquement construites pour des familles de protéines en utilisant l’information dans la base de données BLOCKS (http://blocks.fhcrc.org/). Les familles qu’on utilisera sont les familles des domaines SH3 et PDZ.

Pour leur construction, vous utiliserez l’approche BLOSUM comme expliqué pendant le cours (diapositives de L4: pages 33-­‐48).
Faites attention que pour chaque famille il y a plusieurs BLOCK (4 pour la famille SH3 par exemple). Les valeurs f a,b sont calculées sur les 4 BLOCK indépendamment. Après le f a,b total pour tous les BLOCK ensemble est obtenu en faisant la somme normalisée des ces f a,b par BLOCK.
Pour chaque famille, vous créerez 2 matrices qui sont générées en utilisant des groupements différents: c.-­‐à-­‐d. 70% et 40% d’identité entre les séquences qui font partie du même groupe.
Quand les matrices sont créées, vous expliquez une fois chaque étape de la méthode BLOSUM en utilisant une de ces deux familles comme exemple.

Donnez la possibilité de télécharger les matrices de votre wiki. Examinez aussi la similarité de vos matrices avec la matrice BLOSUM62. Est-ce que les valeurs sur le diagonal sont différent ? Est-ce que certaines substitutions sont maintenant accepté qui n’étaient pas accepté en BLOSUM62 (ou vice versa) ?
Montrez aussi quelques exemples d’alignement pour des séquences de la même famille (en utilisant le logiciel que vous avez implémenté dans le premier mini projet). Est‐ce qu’il y aura des différences entre les alignements quand vous utiliserez des matrices de 70% ou 40% ?

Comparez aussi vos résultats avec les alignements pour les mêmes séquences en utilisant par exemple BLOSUM62. Est-ce que les alignements obtenus en utilisant les matrices que vous avez construites sont meilleurs ?

In [32]:
class Sequence():
    """
    ADT séquence qui représente une séquence d’acides aminés
    et tous les opérations qu’on peut exécuter sur une séquence.
    """
    def __init__(self, sequence):
        self.current = 0
        self.sequence = sequence
    
    def __repr__(self):
        return self.sequence
    
    def __len__(self):
        return len(self.sequence)
    
    def __iter__(self):
        self.current = 0
        return self
    
    def next(self):
        if self.current > len(self.sequence):
            raise StopIteration
        else:
            self.current += 1
            return self.sequence[self.current - 1]
    
    def __getitem__(self, index):
        """
        @desc: Permet d'interpreter la classe comme une "String".
        
        @param{index}: Index de la lettre qu'on veut consulter.
        """
        return self.sequence[index]

    @staticmethod
    def load(filename):
        """
        """
        with open(filename) as f:
            seq = []
            for line in f:
                if line[0] == ">":
                    if seq:
                        yield Sequence(seq)
                    seq = [] 
                else:
                    seq.append(line[:-1]) # Not including '\n'
                    
            if (seq):
                yield Sequence(seq)

In [45]:
def zip_blocks(sequences):
    res = [list() for i in range(len(sequences[0]))]
    for sequence in sequences:
        for i, block in enumerate(sequence):
            res[i].append(block)
            
    # return [item for item in zip(sequences)]
    return res

In [67]:
class SequenceGroup():
    """
    Docstring for SequenceGroup
    """
    def __init__(self, seqList, idPercentage):
        self._percentage = idPercentage
        self._groups = []
        for seq in seqList:
            self._addSequence(seq)

    def _isInSameGroup(self, seq1, seq2):
        """ 
        @desc: Cette méthode verifie si deux séquence sont dans le meme groupe
            càd si elles ont un certain pourcentage d'identité
        """
        similarity = 0
        for i in range(min(len(seq1), len(seq2))):
            if seq1[i] == seq2[i]:
                similarity += 1
                
        result = False
        if similarity > (len(seq1) * (self._percentage / 100)):
            result = True
            
        return result
        
    def _isIngroup(self, seq, group):
        """ 
        @desc: Cette méthode verifie si une séquence appartient à un groupe donné
        """
        for element in group:
            if self._isInSameGroup(seq, element):
                return True
            
        return False

    def _addSequence(self, seq1):
        """ 
        @desc: Ajoute une sequence au bon groupe
        """
        for i, group in enumerate(self._groups):
            if self._isIngroup(seq1, group):
                self._groups[i].append(seq1)
                return

        # Si la séquence n'a été placée nul part.
        self._groups.append([seq1])

    def _getGroupeWeigth(self, groupeId):
        """ 
        @desc: Cette méthode retourne le poids (taille) d'un groupe 
        """
        return len(self._groups[groupeId])

    def _countAminoInGroupCol(self, aminoToCompare, idxGroup, column):
        """ 
        @desc: Cette méthode compte le nombre d'acide aminé dans une 
            colonne d'un groupe
        """
        res = 0
        for seq in self._groups[idxGroup]:
            try:
                if seq[column] == aminoToCompare:
                    res += 1
            except:
                print("Error seq: %s and group: %s" % (seq, self._groups[idxGroup]))
        return res

    def computeF(self, a, b):
        """
        @desc: Cette methode calcule f(a,b) qui correspond à
        """
        fab = 0
        seqSize = len(self._groups[0][0]) # taille d'une sequence
        for idxSeq in range(seqSize):
            for idxGroupe1 in range(len(self._groups)):
                weight1 = self._getGroupeWeigth(idxGroupe1)
                fag1 = self._countAminoInGroupCol(a, idxGroupe1, idxSeq) / weight1
                fbg1 = self._countAminoInGroupCol(b, idxGroupe1, idxSeq) / weight1
                for idxGroupe2 in range(idxGroupe1 + 1, len(self._groups)):
                    weight2 = self._getGroupeWeigth(idxGroupe2)
                    fag2 = self._countAminoInGroupCol(a, idxGroupe2, idxSeq) / weight2
                    fbg2 = self._countAminoInGroupCol(b, idxGroupe2, idxSeq) / weight2 
                    fab += (fag1 * fbg2)
                    if a != b:
                        fab += (fbg1 * fag2)
        return fab

    def __str__(self):
        """ 
        @desc: Represente les groupes de séquence
        """
        for i, groupe in enumerate(self._groups):
            for seq in groupe:
                print("Groupe n°" + str(i) + " : " + str(seq))      

In [68]:
from math import log2

class BlosumMatrix():
    """
    @desc: Represente/calcul une matrix BlosumMatrix
    """
    AMINO = "ARNDCQEGHILKMFPSTWYVBZX"
    def __init__(self, blocks, identity):
        self._blocks = blocks
        self._identity = identity
        self._blocksGroups = list() #liste des groupes
        self._amino = BlosumMatrix.AMINO
        self._matrix = [[0 for i in range(len(self._amino))] for j in range(len(self._amino))]
        self._makeMatrix()
        
    def __repr__(self):
        """
        """
        for idxLine, line in enumerate(self._matrix):
            lineToPrint = self._amino[idxLine] + " "
            for col in line:
                lineToPrint += " " * (4 - len(str(col)))
                lineToPrint += str(col)
            matFile.write(lineToPrint + "\n")
        
    def _createGroups(self):
        """
        @desc: Permet des créer les groupes
        """
        self._blocksGroups = []
        for block in self._blocks:
            group = SequenceGroup(block, self._identity)
            self._blocksGroups.append(group)
            
    def computeFrequency(self):
        """ 
        @desc: Calcul la matrice tel qu'elle contienne les frequence de substitution
        """
        for idxAmino1, amino1 in enumerate(BlosumMatrix.AMINO):
            for idxAmino2, amino2 in enumerate(BlosumMatrix.AMINO):
                fab = 0
                for g in self._blocksGroups:
                    fab += g.computeF(amino1, amino2)
                fab /= len(self._blocks) # normalisation par le nombre de blocs
                self._matrix[idxAmino1][idxAmino2] = fab
    
    def computeProbabilityOccurence(self):
        """
        @desc: Calcule les probabilités d'occurence dans le modèle d'évolution.
        """
        psum = 0 # somme des frequence de substitution tq 1 <= b <= a
        for idxLine, line in enumerate(self._matrix):
            for idxCol, col in enumerate(line):
                if idxLine <= idxCol:
                    psum += col
                    
        for i in range(len(BlosumMatrix.AMINO)):
            for j in range(len(BlosumMatrix.AMINO)):
                self._matrix[i][j] /= psum

    def computeLogChance(self):
        """
        @desc: Calcul des taux de log-chance grace a la probabilite d'occurrence de cet alignement de résidu de a et
            b dans le modèle aléatoire
        """
        dictPsum = dict() # contient les p(a) pour tout a
        for idxAmino1, amino1 in enumerate(BlosumMatrix.AMINO):
            psum = 0
            for idxAmino2, amino2 in enumerate(BlosumMatrix.AMINO):
                if amino1 != amino2:
                    psum += self._matrix[idxAmino1][idxAmino2]
            dictPsum[amino1] = psum
            
        residue = [[0 for i in range(len(BlosumMatrix.AMINO))] for j in range(len(BlosumMatrix.AMINO))]
        
        for a in range(len(BlosumMatrix.AMINO)):
            for b in range(len(BlosumMatrix.AMINO)):
                pa = self._matrix[a][a] + 0.5 * dictPsum[BlosumMatrix.AMINO[a]]
                if a == b :
                    residue[a][a] = pa * pa
                else:
                    pb = self._matrix[b][b] + 0.5 * dictPsum[BlosumMatrix.AMINO[b]]
                    residue[a][b] = 2 * pa * pb
                    
        for idxLine, line in enumerate(self._matrix):
            for idxCol, col in enumerate(line):
                if self._matrix[idxLine][idxCol]!= 0 and residue[idxLine][idxCol] != 0:
                    self._matrix[idxLine][idxCol] = 2 * log2(self._matrix[idxLine][idxCol] / residue[idxLine][idxCol])
                    
        for idxLine, line in enumerate(self._matrix):
            for idxCol, col in enumerate(line):
                self._matrix[idxLine][idxCol] = round(self._matrix[idxLine][idxCol])

    def _makeMatrix(self):
        """
        @desc: Appelle les differentes méthodes qui crée progressivement la matrice 
        """
        self._createGroups()
        self.computeFrequency()
        self.computeProbabilityOccurence()
        self.computeLogChance()

    def toFile(self, filename):
        """
        @desc: Sauvegarde la matrice dans un fichier
        """
        try:
            matFile = open(filename, 'w')
            matFile.write("#  "+ filename + "\n")
            matFile.write("     " + "   ".join(self._amino) + "\n")
            for idxLine, line in enumerate(self._matrix):
                lineToPrint = self._amino[idxLine] + " "
                for col in line:
                    lineToPrint += " " * (4 - len(str(col)))
                    lineToPrint += str(col)
                matFile.write(lineToPrint + "\n")
        except IOError:
            print("Erreur pendant la lecture du fichier : " + filename)

In [69]:
sh2 = zip_blocks([seq.sequence for seq in Sequence.load('./sh2.fasta')])

matrix40 = BlosumMatrix(sh2, 40)
matrix40.toFile("blosum2-sh2.txt")

Error seq: KFDTLWQLVEYLKL and group: ['KFRTLGELVHHHSVH', 'RFNTLAELVHHHSTV', 'RFSTLAELVHHHSTV', 'KFNTLAELVHHHSVP', 'RFNTLAELVHHHSTV', 'RFNTLAELVHHHSTV', 'RFNTLAELVHHHSTV', 'TFPTLQALVQHYSKK', 'QFNSVQELVQHYVEV', 'QFNSVQELVQHYMEV', 'QFDSIQDLVQHYMEV', 'IFSTLNEFVSHYTKT', 'QFETLQQLVQHYSEK', 'QFETLQQLVQHYSER', 'QFETLQQLVQHYSEK', 'QFETLQQLVQHYSER', 'QFDTLQQLVQHYSDR', 'KFNSLNELVDYHRST', 'KFNSLNELVDYHRST', 'RFSSLSDLIGYYSHV', 'RFSSLSDLIGYYSHV', 'RFSSLSDLIGYYSHV', 'TFSTLQELVDHYKKG', 'TFSSLQELVLHYKKG', 'TFSSLQELVVHYKKG', 'KFDTLWQLVEHYSYK', 'KFDTLWQLVEHYSYK', 'NFDTLWQLVEHYSYK', 'KFDTLWQLVEHYSYK', 'TFSSLHELVEYYSSS', 'TFPGLHELVRHYTNA', 'TFPGLHDLVRHYTNA', 'FFCNLMDMVEHYSKD', 'CFCNLMDMVEHYTKD', 'CFCNLMDMVEHYTRD', 'TFTSINEMIQHYQKQ', 'RFQTLGELIGLYAQP', 'IYATLKSLVEHYANN', 'RFNTLAELVHHHSTV', 'QFNSVQELVQHYVEV', 'LYGSLKELVLHYQRT', 'LYSSLKELVLHYQQT', 'YFENLMQLVEHYTND', 'QFQTLLELVHHYEGS', 'FFPTLIELIKHYEKD', 'QFPGPVELVNHHKTS', 'QFGSLQQLVAYYSKH', 'RFNTLAELVHHHSTV', 'QFSSLQQLVAYYSKH', 'PFPSLPELVRHYQGK', 'CFCNLMDMVEH